In [1]:
from line_solver import *

In [2]:
model = Network('model')

P1 = Place(model, 'P1')
P2 = Place(model, 'P2')
P3 = Place(model, 'P3')
P4 = Place(model, 'P4')

T1 = Transition(model, 'T1')
T2 = Transition(model, 'T2')
T3 = Transition(model, 'T3')
T4 = Transition(model, 'T4')

jobclass = ClosedClass(model, 'Class1', 2, P1, 0)

In [3]:
# T1 - Exponential distribution
mode1 = T1.add_mode('Mode1')
T1.set_distribution(mode1, Exp(2))
T1.set_enabling_conditions(mode1, jobclass, P1, 2)
T1.set_firing_outcome(mode1, jobclass, P2, 2)

# T2 - Erlang distribution (shape=3, rate=4)
mode2 = T2.add_mode('Mode2')
T2.set_distribution(mode2, Erlang(3, 4))
T2.set_enabling_conditions(mode2, jobclass, P2, 2)
T2.set_firing_outcome(mode2, jobclass, P3, 2)

# T3 - Hyperexponential distribution (p=0.7, lambda1=3, lambda2=1.5)
mode3 = T3.add_mode('Mode3')
T3.set_distribution(mode3, HyperExp(0.7, 3, 1.5))
T3.set_enabling_conditions(mode3, jobclass, P3, 2)
T3.set_firing_outcome(mode3, jobclass, P4, 2)

# T4 - Coxian distribution with custom parameters
# Define Coxian parameters: mu (service rates) and phi (transition probabilities)
mu0 = np.array([[1.0], [2.0]])  # Service rates for each phase
phi0 = np.array([[0.6], [1.0]]) # Transition probabilities

mode4 = T4.add_mode('Mode4')
T4.set_distribution(mode4, Coxian(mu0, phi0))
T4.set_enabling_conditions(mode4, jobclass, P4, 2)
T4.set_firing_outcome(mode4, jobclass, P1, 2)

In [4]:
# Set up routing matrix for serial flow
routingMatrix = model.init_routing_matrix()
routingMatrix.set(jobclass, jobclass, P1, T1, 1.0)  # P1 -> T1
routingMatrix.set(jobclass, jobclass, P2, T2, 1.0)  # P2 -> T2
routingMatrix.set(jobclass, jobclass, P3, T3, 1.0)  # P3 -> T3
routingMatrix.set(jobclass, jobclass, P4, T4, 1.0)  # P4 -> T4

routingMatrix.set(jobclass, jobclass, T1, P2, 1.0)  # T1 -> P2
routingMatrix.set(jobclass, jobclass, T2, P3, 1.0)  # T2 -> P3
routingMatrix.set(jobclass, jobclass, T3, P4, 1.0)  # T3 -> P4
routingMatrix.set(jobclass, jobclass, T4, P1, 1.0)  # T4 -> P1 (back to start)

model.link(routingMatrix)

In [5]:
# Set initial state - all 2 tokens start in P1
P1.set_state(np.array([[jobclass.get_population()]]))
P2.set_state(np.array([[0]]))
P3.set_state(np.array([[0]]))
P4.set_state(np.array([[0]]))

In [6]:
# Solve the model
solver = JMT(model, seed=23000)
avgTable = solver.avg_table()

JMT Model: /tmp/workspace/jsim/7906216698791189964/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.207335s.
  Station JobClass    QLen  Util   RespT  ResidT    ArvR    Tput
0      P1   Class1  0.2775   0.0  0.4822  0.4822  0.5746  0.5743
1      P2   Class1  0.7768   0.0  1.3441  1.3441  0.5743  0.5727
2      P3   Class1  0.2516   0.0  0.4304  0.4304  0.5727  0.5729
3      P4   Class1  0.6939   0.0  1.2301  1.2301  0.5729  0.5744
